In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import tools as tools

import pickle
import numpy as np

In [2]:
'''import data - parents'''
technical_team = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/technical_team.csv')
cross_cutting_programme_1 = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/cross_cutting_programme_1.csv')
cross_cutting_programme_2 = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/cross_cutting_programme_2.csv')

'''import data - technical areas'''
technical_area_1 = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/technical_area_1.csv')
technical_area_2 = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/technical_area_2.csv')

'''import data - technical areas sub categories'''
technical_sub_area_1 = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/technical_sub_area_1.csv')
technical_sub_area_2 = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/technical_sub_area_2.csv')

In [3]:
'''count and re-process labels'''
technical_team, df_lambda_ms_1 = tools.count_labels(technical_team)
cross_cutting_programme_1, df_lambda_ms_2 = tools.count_labels(cross_cutting_programme_1)
cross_cutting_programme_2, df_lambda_ms_3 = tools.count_labels(cross_cutting_programme_2)

Shape:(7, 2)


,category,Count
0,chemicals_and_waste_programme,38
1,climate_change_adaptation_programme,96
2,climate_forest_of_climate_change_mitigation,3
3,climate_strategies_and_policy_programme,102
4,ecosystems_and_biodiversity_programme,189
5,energy_program_of_climate_change_mitigation,107
6,oceans_and_water_programme,41


Shape:(4, 2)


,category,Count
0,climate_change_adaptation_programme,54
1,ecosystems_and_biodiversity_programme,40
2,environmental_governance_programme,5
3,no tag,477


Shape:(6, 2)


,category,Count
0,chemicals_and_waste_programme,1
1,climate_change_adaptation_programme,9
2,climate_forest_of_climate_change_mitigation,25
3,energy_program_of_climate_change_mitigation,11
4,environmental_governance_programme,1
5,no tag,529


## First create data set that includes technical_team and cross-cutting programme 1&2 to build a TT classifier. Following landscape, strategy approach

In [4]:
# delete rows with non-existing categories:
teams = ['chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_strategies_and_policy_programme',
       'ecosystems_and_biodiversity_programme',
       'energy_program_of_climate_change_mitigation',
       'oceans_and_water_programme', 'environmental_governance_programme'] #environmental_governance_programme added

technical_team = technical_team[technical_team['category'].isin(teams)]
cross_cutting_programme_1 = cross_cutting_programme_1[cross_cutting_programme_1['category'].isin(teams)]
cross_cutting_programme_2 = cross_cutting_programme_2[cross_cutting_programme_2['category'].isin(teams)]


#add missing column:
technical_team['environmental_governance_programme'] = 0

In [5]:
# turn to multi_label problem
relevant = technical_team[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_strategies_and_policy_programme',
       'ecosystems_and_biodiversity_programme',
       'energy_program_of_climate_change_mitigation',
       'oceans_and_water_programme', 'environmental_governance_programme', 'category']]

relevant.rename({'category':'category_1'}, axis = 1, inplace = True)
              
result = pd.concat([relevant, cross_cutting_programme_1['category']], axis=1, sort=False)
result.rename({'category':'category_2'}, axis = 1, inplace = True)

result = pd.concat([result, cross_cutting_programme_2['category']], axis=1, sort=False)
result.rename({'category':'category_3'}, axis = 1, inplace = True)


result[['chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_strategies_and_policy_programme',
       'ecosystems_and_biodiversity_programme',
       'energy_program_of_climate_change_mitigation',
       'oceans_and_water_programme', 'environmental_governance_programme']] = result[['chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_strategies_and_policy_programme',
       'ecosystems_and_biodiversity_programme',
       'energy_program_of_climate_change_mitigation',
       'oceans_and_water_programme', 'environmental_governance_programme']].fillna(0).astype(int)

def to_multi_label(dataframe):
    for index, row in dataframe.iterrows():

        for column in dataframe.columns:

            if row['category_2'] == column:

                if row[column] == 0:
                    dataframe.at[index, column] = 1
            else:
                pass

            if row['category_3'] == column:

                if row[column] == 0:
                    dataframe.at[index, column] = 1
            else:
                pass 
    
    
    dataframe['labels']= dataframe[['chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_strategies_and_policy_programme',
       'ecosystems_and_biodiversity_programme',
       'energy_program_of_climate_change_mitigation',
       'oceans_and_water_programme', 'environmental_governance_programme']].fillna(0).astype(int).values.tolist()
    
    return dataframe


to_multi_label(result)
result = result.reset_index(drop=True)

result.to_json(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/technical_team_all.json")

## structure dataset to areas and sub areas

In [6]:
'''Technical Areas:'''

ecosystems_and_biodiversity_programme = ['ecosystem_management_and_restoration', 'mainstreaming_biodiversity',
                                        'strengthening_conservation_areas']

chemicals_and_waste_programme = ['persistent_organic_pollutants', 'ozone_depleting_substances', 'heavy_metals', 
                                 'waste_management', 'cooling_alternatives']

energy_program_of_climate_change_mitigation = ['energy_efficiency', 'renewable_energy', 'energy_access', 
                                               'miscellaneous_energy_program']


oceans_and_water_programme = ['small_island_dev_states_program_sids', 'climate_resilient_access_to_water_supply_sanitation',
                             'climate_resilient_integrated_water_resource_coastal_area_management', 
                              'cross_cutting_governance_and_learning', 
                             'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
                             'sustainable_management_of_oceans_in_a_changing_climate']

climate_forest_of_climate_change_mitigation = ['carbon_conservation','carbon_sequestration', 'climate_financing_for_redd_redd']

climate_change_adaptation_programme =['climate_information_and_early_warning_systems_ci_ews', 
                                      'cross_sectoral_climate_resilient_livelihoods',
                                      'fostering_food_security_and_resilient_agricultural_systems',
                                     'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
                                     'ecosystem_based_adaptation',
                                      'urban_resilience',
                                     'climate_resilient_integrated_water_resource_and_coastal_management']

environmental_governance_programme = ['mainstreaming_green_economy', 'gcp_green_commodities_programme']

climate_strategies_and_policy_programme = []

a = technical_area_2.columns.to_list()
b = [item for item in a if item not in ecosystems_and_biodiversity_programme]
b = [item for item in b if item not in energy_program_of_climate_change_mitigation]
b = [item for item in b if item not in chemicals_and_waste_programme]
b = [item for item in b if item not in oceans_and_water_programme]
b = [item for item in b if item not in climate_change_adaptation_programme]
b = [item for item in b if item not in environmental_governance_programme]
b = [item for item in b if item not in climate_strategies_and_policy_programme]
b = [item for item in b if item not in climate_forest_of_climate_change_mitigation]

'''sub categories'''
#b = [item for item in b if item not in energy_efficiency]
#b = [item for item in b if item not in renewable_energy]
#b = [item for item in b if item not in ecosystem_management_and_restoration]
#b = [item for item in b if item not in energy_access]
#b = [item for item in b if item not in miscellaneous_energy_program]

b
#need list of sub categories per parent

['PIMS_ID',
 'all_text_clean',
 'all_text_clean_spacy',
 'technical_area_2',
 'Unnamed: 4',
 'no tag']

## clean up sub-areas:

In [7]:
'''Ecosystem and Biodiversity Programme Sub-Areas:'''
ecosystem_management_and_restoration = [ '_access_and_benefit_sharing', 
                                        '_agrobiodiversity', '_ecosystem_based_adaptation',
                                        '_ecosystem_based_mitigation',  '_biodiversity_financing', 
                                        '_wildlife_conservation' ] # clean up first



'''Energy Programme of Climate Change Mitigaion Sub-Areas:'''
energy_efficiency = ['_appliances', '_agriculture_irrigation', '_buildings', '_cooling', '_emobility', '_industrial',
                     '_lighting', '_transport', '_e_mobility', ]




renewable_energy = ['_type_off_grid_mini_grids',
 '_type_off_grid_standalone_systems',
 '_type_on_grid_rooftop',
 '_type_on_grid_utility_scale',
 '_type_thermal', '_resource_battery','_resource_biomass',
 '_resource_diesel_hybrid',
 '_resource_geothermal',
 '_resource_hydro',
 '_resource_solar',
 '_resource_waste_agriculture',
 '_resource_waste_municipal',
 '_resource_wind',]

energy_access = ['_clean_cooking', '_sustainable_charcoal', '_electricity_access']


miscellaneous_energy_program = ['_crisis', '_digital', '_enabling_activities', '_fossil_fuel_subsidy_reform', '_tourism',
                               '_other_energy_projects', '_sustainable_cities'] # no health, carbon pricing


'''Chemical Waste:'''

persistent_organic_pollutants = ['_pesticide','_flame_retardants', 
                                 '_unintentional_pops_e_g_furans_dioxins', '_pcbs'] # clean up duplicate

ozone_depleting_substances = ['_hcfcs']

heavy_metals = [ '_mercury', '_lead', '_source_from_asgm', '_source_from_health_sector', '_source_from_industrial_processes']

waste_management = ['_hazardous_chemical_waste', '_hazardous_chemical_waste_e_g_legacy_waste', '_e_waste']

cooling_alternatives = ['_natural_refrigerants_co2_ammonia_propane_isobutane', '_not_in_kind_technology']




sub1 = technical_sub_area_2.columns.to_list()

a= [item for item in sub1 if item not in ecosystem_management_and_restoration]
a= [item for item in a if item not in energy_efficiency]
a= [item for item in a if item not in persistent_organic_pollutants]
a= [item for item in a if item not in ozone_depleting_substances]
a= [item for item in a if item not in heavy_metals]
a= [item for item in a if item not in waste_management]
a= [item for item in a if item not in cooling_alternatives]
a= [item for item in a if item not in renewable_energy]
a= [item for item in a if item not in energy_access]
a= [item for item in a if item not in miscellaneous_energy_program]

a

['PIMS_ID',
 'all_text_clean',
 'all_text_clean_spacy',
 'technical_sub_area_2',
 'no tag']

In [8]:
result.columns

Index(['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_strategies_and_policy_programme',
       'ecosystems_and_biodiversity_programme',
       'energy_program_of_climate_change_mitigation',
       'oceans_and_water_programme', 'environmental_governance_programme',
       'category_1', 'category_2', 'category_3', 'labels'],
      dtype='object')

In [9]:
# count number of categories: 
agg = pd.DataFrame(result[['chemicals_and_waste_programme', 'climate_change_adaptation_programme',
       'climate_forest_of_climate_change_mitigation',
       'climate_strategies_and_policy_programme',
       'ecosystems_and_biodiversity_programme',
       'energy_program_of_climate_change_mitigation',
       'oceans_and_water_programme', 'environmental_governance_programme',]].sum(axis=0))
d = agg[0].T.to_dict()

d

{'chemicals_and_waste_programme': 39,
 'climate_change_adaptation_programme': 159,
 'climate_forest_of_climate_change_mitigation': 28,
 'climate_strategies_and_policy_programme': 102,
 'ecosystems_and_biodiversity_programme': 229,
 'energy_program_of_climate_change_mitigation': 118,
 'oceans_and_water_programme': 41,
 'environmental_governance_programme': 6}

In [10]:
Chemical_Waste = result[result.chemicals_and_waste_programme == 1]
CC_Adaption = result[result.climate_change_adaptation_programme == 1]
Climate_Forest = result[result.climate_forest_of_climate_change_mitigation == 1]
Climate_Strategies = result[result.climate_strategies_and_policy_programme == 1]
Ecosystems_Biodiversity = result[result.ecosystems_and_biodiversity_programme == 1]
Energy_Programme = result[result.energy_program_of_climate_change_mitigation == 1]
Oceans_Water = result[result.oceans_and_water_programme == 1]
Environmental_Governance = result[result.environmental_governance_programme == 1]

## Process sub categories

In [11]:
df_org = result[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy']]
df_num_1 = technical_area_1._get_numeric_data()
df_num_1_1 = df_num_1.drop(columns=['no tag'])
df_num_2 = technical_area_2._get_numeric_data()
df_num_2_1 = df_num_2.drop(columns=['no tag'])


'''combine lists'''
df_num_1 = df_num_1.drop(columns=['PIMS_ID','no tag'])
df_num_2 = df_num_2.drop(columns=['PIMS_ID','no tag'])
sub_sub_1 = df_num_1.columns.tolist()
sub_sub_2 = df_num_2.columns.tolist()
sub_sub = sub_sub_1 + list(set(sub_sub_2) - set(sub_sub_1))

unmatched_items_1 = [d for d in sub_sub_1 if d not in sub_sub_2]
unmatched_items_2 = [d for d in sub_sub_2 if d not in sub_sub_1]

for col in unmatched_items_1:
    df_num_2_1[col] = 0
    
for col in unmatched_items_2:
    df_num_1_1[col] = 0
    
sub_category = df_num_1_1.merge(df_num_2_1, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_category.iterrows():
    for i in sub_sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_category.at[index, i] = 1
        else:
            sub_category.at[index, i] = 0     
            
for i in sub_sub:
    sub_category= sub_category.drop(columns=[i+"_x"])
    sub_category= sub_category.drop(columns=[i+"_y"])
    
sub_category = sub_category.drop(columns=['PIMS_ID'])

#sub_technical_team = pd.concat([df_org, sub_category], axis=1)

## Process sub-sub categories

In [15]:
df_org = result[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy']]
df_num_1 = technical_sub_area_1._get_numeric_data()
df_num_1_1 = df_num_1.drop(columns=['no tag'])
df_num_2 = technical_sub_area_2._get_numeric_data()
df_num_2_1 = df_num_2.drop(columns=['no tag'])


'''combine lists'''
df_num_1 = df_num_1.drop(columns=['PIMS_ID','no tag'])
df_num_2 = df_num_2.drop(columns=['PIMS_ID','no tag'])
sub_sub_1 = df_num_1.columns.tolist()
sub_sub_2 = df_num_2.columns.tolist()
sub_sub = sub_sub_1 + list(set(sub_sub_2) - set(sub_sub_1))

unmatched_items_1 = [d for d in sub_sub_1 if d not in sub_sub_2]
unmatched_items_2 = [d for d in sub_sub_2 if d not in sub_sub_1]

for col in unmatched_items_1:
    df_num_2_1[col] = 0
    
for col in unmatched_items_2:
    df_num_1_1[col] = 0
    
sub_sub_category = df_num_1_1.merge(df_num_2_1, how='left', left_on=['PIMS_ID'], right_on=['PIMS_ID'])

for index, row in sub_sub_category.iterrows():
    for i in sub_sub:          
        if row[i+"_x"] or row[i+"_y"]:
            sub_sub_category.at[index, i] = 1
        else:
            sub_sub_category.at[index, i] = 0     
            
for i in sub_sub:
    sub_sub_category= sub_sub_category.drop(columns=[i+"_x"])
    sub_sub_category= sub_sub_category.drop(columns=[i+"_y"])
    
sub_sub_category = sub_sub_category.drop(columns=['PIMS_ID'])
#sub_sub_technical_team = pd.concat([df_org, sub_category], axis=1)
#sub_sub_technical_team

## Parents and merge

In [16]:
technical_team = result.drop(columns=['category_1', 'category_2', 'category_3', 'labels'])
technical_team = pd.concat([technical_team, sub_category], axis=1)
technical_team = pd.concat([technical_team, sub_sub_category], axis=1)


# rename until cleaned!
technical_team = technical_team.rename(columns={'waste_management': 'chemical_waste_management'})

technical_team.to_csv(os.path.abspath(os.path.join('../..', 'data/processed/'))+"/technical_team.csv")

## Create dict

In [17]:
technical_teams = ['ecosystems_and_biodiversity_programme',
'chemicals_and_waste_programme',
'energy_program_of_climate_change_mitigation',
'oceans_and_water_programme',
'climate_forest_of_climate_change_mitigation',
'climate_change_adaptation_programme' ,
'environmental_governance_programme' ,
'climate_strategies_and_policy_programme']

#chemicals_and_waste
ecosystems_and_biodiversity_programme = []
chemicals_and_waste_programme = []
energy_program_of_climate_change_mitigation = []
oceans_and_water_programme = []
climate_forest_of_climate_change_mitigation = []
climate_change_adaptation_programme =[]
environmental_governance_programme = []
climate_strategies_and_policy_programme = []

lists = [ecosystems_and_biodiversity_programme,
chemicals_and_waste_programme,
energy_program_of_climate_change_mitigation,
oceans_and_water_programme,
climate_forest_of_climate_change_mitigation,
climate_change_adaptation_programme,
environmental_governance_programme,
climate_strategies_and_policy_programme,]

d = dict(zip(technical_teams, lists))

In [18]:
for key, value in d.items():
    for index, row in technical_team.iterrows():
        if row[key] == 1:
            for i in range(len(technical_team.columns)): 
                if row[i] == 1:
                    if row.index[i] not in value:
                        value.append(row.index[i])
                        
for key, value in d.copy().items():
    for i in value:
        if i == key:
            print(i)
            value.remove(i)
            
with open(os.path.abspath(os.path.join('../..', 'data/processed/'))+"/technical_team_dict.pkl", 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

ecosystems_and_biodiversity_programme
chemicals_and_waste_programme
energy_program_of_climate_change_mitigation
oceans_and_water_programme
climate_forest_of_climate_change_mitigation
climate_change_adaptation_programme
environmental_governance_programme
climate_strategies_and_policy_programme


## Chemical Waste

In [82]:
chemicals_and_waste_programme = ['persistent_organic_pollutants', 'ozone_depleting_substances', 'heavy_metals', 
                                 'waste_management', 'cooling_alternatives']

#lets merge with the subcategories to obtain a dataframe that contains all subcategores for parent category:
Chemical_Waste = Chemical_Waste[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'chemicals_and_waste_programme']]

area_1 = technical_area_1[['PIMS_ID', 'persistent_organic_pollutants', 'ozone_depleting_substances', 'heavy_metals', 
                                 'waste_management']]

area_2 = technical_area_2[['PIMS_ID', 'persistent_organic_pollutants',  'heavy_metals', 
                                 'waste_management', 'cooling_alternatives']]

area_1['cooling_alternatives'] = 0
area_2['ozone_depleting_substances'] = 0


Chemical_Waste = Chemical_Waste.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Chemical_Waste = Chemical_Waste.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')


In [140]:
chemicals_and_waste_programme = ['persistent_organic_pollutants', 'ozone_depleting_substances', 'heavy_metals', 
                                 'waste_management', 'cooling_alternatives']

#lets merge with the subcategories to obtain a dataframe that contains all subcategores for parent category:
Chemical_Waste = Chemical_Waste[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'chemicals_and_waste_programme']]

area_1 = technical_area_1[['PIMS_ID', 'persistent_organic_pollutants', 'ozone_depleting_substances', 'heavy_metals', 
                                 'waste_management']]

area_2 = technical_area_2[['PIMS_ID', 'persistent_organic_pollutants',  'heavy_metals', 
                                 'waste_management', 'cooling_alternatives']]

area_1['cooling_alternatives'] = 0
area_2['ozone_depleting_substances'] = 0


Chemical_Waste = Chemical_Waste.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Chemical_Waste = Chemical_Waste.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Chemical_Waste.iterrows():
    for i in chemicals_and_waste_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Chemical_Waste.at[index, i] = 1
        else:
            Chemical_Waste.at[index, i] = 0            
for i in chemicals_and_waste_programme:
    Chemical_Waste= Chemical_Waste.drop(columns=[i+"_x"])
    Chemical_Waste= Chemical_Waste.drop(columns=[i+"_y"])

Chemical_Waste.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/chemical_waste.xlsx")


'''full data'''
Chemical_Waste = result
chemicals_and_waste_programme = ['persistent_organic_pollutants', 'ozone_depleting_substances', 'heavy_metals', 
                                 'waste_management', 'cooling_alternatives']

#lets merge with the subcategories to obtain a dataframe that contains all subcategores for parent category:
Chemical_Waste = Chemical_Waste[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'chemicals_and_waste_programme']]

area_1 = technical_area_1[['PIMS_ID', 'persistent_organic_pollutants', 'ozone_depleting_substances', 'heavy_metals', 
                                 'waste_management']]

area_2 = technical_area_2[['PIMS_ID', 'persistent_organic_pollutants',  'heavy_metals', 
                                 'waste_management', 'cooling_alternatives']]

area_1['cooling_alternatives'] = 0
area_2['ozone_depleting_substances'] = 0


Chemical_Waste = Chemical_Waste.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Chemical_Waste = Chemical_Waste.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Chemical_Waste.iterrows():
    for i in chemicals_and_waste_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Chemical_Waste.at[index, i] = 1
        else:
            Chemical_Waste.at[index, i] = 0            
for i in chemicals_and_waste_programme:
    Chemical_Waste= Chemical_Waste.drop(columns=[i+"_x"])
    Chemical_Waste= Chemical_Waste.drop(columns=[i+"_y"])

Chemical_Waste.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/chemical_waste_full.xlsx")

# Climate Change Adaption

In [141]:
climate_change_adaptation_programme = ['climate_information_and_early_warning_systems_ci_ews',
 'cross_sectoral_climate_resilient_livelihoods',
 'fostering_food_security_and_resilient_agricultural_systems',
 'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
 'ecosystem_based_adaptation',
 'urban_resilience',
 'climate_resilient_integrated_water_resource_and_coastal_management']

CC_Adaption = CC_Adaption[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'climate_change_adaptation_programme']]

area_1 = technical_area_1[['PIMS_ID', 'climate_information_and_early_warning_systems_ci_ews',
 'cross_sectoral_climate_resilient_livelihoods',
 'fostering_food_security_and_resilient_agricultural_systems',
 'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
 'ecosystem_based_adaptation',
 'climate_resilient_integrated_water_resource_and_coastal_management']]

area_2 = technical_area_2[['PIMS_ID', 'climate_information_and_early_warning_systems_ci_ews',
 'cross_sectoral_climate_resilient_livelihoods',
 'fostering_food_security_and_resilient_agricultural_systems',
 'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
 'ecosystem_based_adaptation',
 'urban_resilience']]

area_1['urban_resilience'] = 0
area_2['climate_resilient_integrated_water_resource_and_coastal_management'] = 0

CC_Adaption = CC_Adaption.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
CC_Adaption = CC_Adaption.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in CC_Adaption.iterrows():
    for i in climate_change_adaptation_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            CC_Adaption.at[index, i] = 1
        else:
            CC_Adaption.at[index, i] = 0            
for i in climate_change_adaptation_programme:
    CC_Adaption= CC_Adaption.drop(columns=[i+"_x"])
    CC_Adaption= CC_Adaption.drop(columns=[i+"_y"])

CC_Adaption.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/cc_adaption.xlsx")

'''full data'''
CC_Adaption = result

CC_Adaption = CC_Adaption[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'climate_change_adaptation_programme']]

area_1 = technical_area_1[['PIMS_ID', 'climate_information_and_early_warning_systems_ci_ews',
 'cross_sectoral_climate_resilient_livelihoods',
 'fostering_food_security_and_resilient_agricultural_systems',
 'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
 'ecosystem_based_adaptation',
 'climate_resilient_integrated_water_resource_and_coastal_management']]

area_2 = technical_area_2[['PIMS_ID', 'climate_information_and_early_warning_systems_ci_ews',
 'cross_sectoral_climate_resilient_livelihoods',
 'fostering_food_security_and_resilient_agricultural_systems',
 'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
 'ecosystem_based_adaptation',
 'urban_resilience']]

area_1['urban_resilience'] = 0
area_2['climate_resilient_integrated_water_resource_and_coastal_management'] = 0

CC_Adaption = CC_Adaption.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
CC_Adaption = CC_Adaption.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in CC_Adaption.iterrows():
    for i in climate_change_adaptation_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            CC_Adaption.at[index, i] = 1
        else:
            CC_Adaption.at[index, i] = 0            
for i in climate_change_adaptation_programme:
    CC_Adaption= CC_Adaption.drop(columns=[i+"_x"])
    CC_Adaption= CC_Adaption.drop(columns=[i+"_y"])

CC_Adaption.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/cc_adaption_full.xlsx")


## Climate Forest

In [142]:
climate_forest_of_climate_change_mitigation = ['carbon_sequestration', 'climate_financing_for_redd_redd']

In [143]:
Climate_Forest = Climate_Forest[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'climate_forest_of_climate_change_mitigation']]

area_1 = technical_area_1[['PIMS_ID', 'carbon_sequestration', 'climate_financing_for_redd_redd']]

area_2 = technical_area_2[['PIMS_ID', 'carbon_sequestration']]

#area_1['urban_resilience'] = 0
area_2['climate_financing_for_redd_redd'] = 0

Climate_Forest = Climate_Forest.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Climate_Forest = Climate_Forest.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Climate_Forest.iterrows():
    for i in climate_forest_of_climate_change_mitigation:
        
        if row[i+"_x"] or row[i+"_y"]:
            Climate_Forest.at[index, i] = 1
        else:
            Climate_Forest.at[index, i] = 0            
for i in climate_forest_of_climate_change_mitigation:
    Climate_Forest= Climate_Forest.drop(columns=[i+"_x"])
    Climate_Forest= Climate_Forest.drop(columns=[i+"_y"])

Climate_Forest.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/climate_forest.xlsx")

'''full data'''
Climate_Forest = result

Climate_Forest = Climate_Forest[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'climate_forest_of_climate_change_mitigation']]

area_1 = technical_area_1[['PIMS_ID', 'carbon_sequestration', 'climate_financing_for_redd_redd']]

area_2 = technical_area_2[['PIMS_ID', 'carbon_sequestration']]

#area_1['urban_resilience'] = 0
area_2['climate_financing_for_redd_redd'] = 0

Climate_Forest = Climate_Forest.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Climate_Forest = Climate_Forest.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Climate_Forest.iterrows():
    for i in climate_forest_of_climate_change_mitigation:
        
        if row[i+"_x"] or row[i+"_y"]:
            Climate_Forest.at[index, i] = 1
        else:
            Climate_Forest.at[index, i] = 0            
for i in climate_forest_of_climate_change_mitigation:
    Climate_Forest= Climate_Forest.drop(columns=[i+"_x"])
    Climate_Forest= Climate_Forest.drop(columns=[i+"_y"])

Climate_Forest.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/climate_forest_full.xlsx")

## Climate Strategies

In [158]:
Climate_Strategies = result[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy','climate_strategies_and_policy_programme']]

## Ecosystems & Biodiversity

In [159]:
ecosystems_and_biodiversity_programme = ['ecosystem_management_and_restoration',
 'mainstreaming_biodiversity',
 'strengthening_conservation_areas']

In [160]:
Ecosystems_Biodiversity = Ecosystems_Biodiversity[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'ecosystems_and_biodiversity_programme']]

area_1 = technical_area_1[['PIMS_ID', 'ecosystem_management_and_restoration',
 'mainstreaming_biodiversity',
 'strengthening_conservation_areas']]

area_2 = technical_area_2[['PIMS_ID', 'ecosystem_management_and_restoration',
 'mainstreaming_biodiversity',
 'strengthening_conservation_areas']]

#area_1['urban_resilience'] = 0
#area_2['climate_financing_for_redd_redd'] = 0

Ecosystems_Biodiversity = Ecosystems_Biodiversity.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Ecosystems_Biodiversity = Ecosystems_Biodiversity.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Ecosystems_Biodiversity.iterrows():
    for i in ecosystems_and_biodiversity_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Ecosystems_Biodiversity.at[index, i] = 1
        else:
            Ecosystems_Biodiversity.at[index, i] = 0            
for i in ecosystems_and_biodiversity_programme:
    Ecosystems_Biodiversity= Ecosystems_Biodiversity.drop(columns=[i+"_x"])
    Ecosystems_Biodiversity= Ecosystems_Biodiversity.drop(columns=[i+"_y"])

Ecosystems_Biodiversity.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/ecosystem_biodiversity.xlsx")

'''full data'''
Ecosystems_Biodiversity = result

Ecosystems_Biodiversity = Ecosystems_Biodiversity[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'ecosystems_and_biodiversity_programme']]

area_1 = technical_area_1[['PIMS_ID', 'ecosystem_management_and_restoration',
 'mainstreaming_biodiversity',
 'strengthening_conservation_areas']]

area_2 = technical_area_2[['PIMS_ID', 'ecosystem_management_and_restoration',
 'mainstreaming_biodiversity',
 'strengthening_conservation_areas']]

#area_1['urban_resilience'] = 0
#area_2['climate_financing_for_redd_redd'] = 0

Ecosystems_Biodiversity = Ecosystems_Biodiversity.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Ecosystems_Biodiversity = Ecosystems_Biodiversity.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Ecosystems_Biodiversity.iterrows():
    for i in ecosystems_and_biodiversity_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Ecosystems_Biodiversity.at[index, i] = 1
        else:
            Ecosystems_Biodiversity.at[index, i] = 0            
for i in ecosystems_and_biodiversity_programme:
    Ecosystems_Biodiversity= Ecosystems_Biodiversity.drop(columns=[i+"_x"])
    Ecosystems_Biodiversity= Ecosystems_Biodiversity.drop(columns=[i+"_y"])

Ecosystems_Biodiversity.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/ecosystem_biodiversity_full.xlsx")


## Energy Programme

In [161]:
energy_program_of_climate_change_mitigation = ['energy_efficiency',
 'renewable_energy',
 'energy_access',
 'miscellaneous_energy_program']

In [162]:
Energy_Programme = Energy_Programme[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'energy_program_of_climate_change_mitigation']]

area_1 = technical_area_1[['PIMS_ID', 'energy_efficiency',
 'renewable_energy',
 'energy_access',
 'miscellaneous_energy_program']]

area_2 = technical_area_2[['PIMS_ID', 'energy_efficiency',
 'renewable_energy',
 'energy_access',
 'miscellaneous_energy_program']]

#area_1['urban_resilience'] = 0
#area_2['climate_financing_for_redd_redd'] = 0

Energy_Programme = Energy_Programme.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Energy_Programme = Energy_Programme.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Energy_Programme.iterrows():
    for i in energy_program_of_climate_change_mitigation:
        
        if row[i+"_x"] or row[i+"_y"]:
            Energy_Programme.at[index, i] = 1
        else:
            Energy_Programme.at[index, i] = 0            
for i in energy_program_of_climate_change_mitigation:
    Energy_Programme= Energy_Programme.drop(columns=[i+"_x"])
    Energy_Programme= Energy_Programme.drop(columns=[i+"_y"])

Energy_Programme.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/energy_programme.xlsx")

'''full data'''
Energy_Programme = result

Energy_Programme = Energy_Programme[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'energy_program_of_climate_change_mitigation']]

area_1 = technical_area_1[['PIMS_ID', 'energy_efficiency',
 'renewable_energy',
 'energy_access',
 'miscellaneous_energy_program']]

area_2 = technical_area_2[['PIMS_ID', 'energy_efficiency',
 'renewable_energy',
 'energy_access',
 'miscellaneous_energy_program']]

#area_1['urban_resilience'] = 0
#area_2['climate_financing_for_redd_redd'] = 0

Energy_Programme = Energy_Programme.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Energy_Programme = Energy_Programme.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Energy_Programme.iterrows():
    for i in energy_program_of_climate_change_mitigation:
        
        if row[i+"_x"] or row[i+"_y"]:
            Energy_Programme.at[index, i] = 1
        else:
            Energy_Programme.at[index, i] = 0            
for i in energy_program_of_climate_change_mitigation:
    Energy_Programme= Energy_Programme.drop(columns=[i+"_x"])
    Energy_Programme= Energy_Programme.drop(columns=[i+"_y"])

Energy_Programme.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/energy_programme_full.xlsx")

## Oceans and Water

In [163]:
oceans_and_water_programme = ['small_island_dev_states_program_sids',
 'climate_resilient_access_to_water_supply_sanitation',
 'climate_resilient_integrated_water_resource_coastal_area_management',
 'cross_cutting_governance_and_learning',
 'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
 'sustainable_management_of_oceans_in_a_changing_climate']

In [164]:
Oceans_Water = Oceans_Water[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'oceans_and_water_programme']]

area_1 = technical_area_1[['PIMS_ID', 'small_island_dev_states_program_sids',
 'climate_resilient_access_to_water_supply_sanitation',
 'climate_resilient_integrated_water_resource_coastal_area_management',
 'cross_cutting_governance_and_learning',
 'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
 'sustainable_management_of_oceans_in_a_changing_climate']]

area_2 = technical_area_2[['PIMS_ID',
 'cross_cutting_governance_and_learning',
 'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
 'sustainable_management_of_oceans_in_a_changing_climate']]

#area_1['urban_resilience'] = 0
area_2['small_island_dev_states_program_sids'] = 0
area_2['climate_resilient_access_to_water_supply_sanitation'] = 0
area_2['climate_resilient_integrated_water_resource_coastal_area_management'] = 0

Oceans_Water = Oceans_Water.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Oceans_Water = Oceans_Water.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Oceans_Water.iterrows():
    for i in oceans_and_water_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Oceans_Water.at[index, i] = 1
        else:
            Oceans_Water.at[index, i] = 0            
for i in oceans_and_water_programme:
    Oceans_Water= Oceans_Water.drop(columns=[i+"_x"])
    Oceans_Water= Oceans_Water.drop(columns=[i+"_y"])

Oceans_Water.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/oceans_water.xlsx")

'''full data'''
Oceans_Water = result

Oceans_Water = Oceans_Water[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'oceans_and_water_programme']]

area_1 = technical_area_1[['PIMS_ID', 'small_island_dev_states_program_sids',
 'climate_resilient_access_to_water_supply_sanitation',
 'climate_resilient_integrated_water_resource_coastal_area_management',
 'cross_cutting_governance_and_learning',
 'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
 'sustainable_management_of_oceans_in_a_changing_climate']]

area_2 = technical_area_2[['PIMS_ID',
 'cross_cutting_governance_and_learning',
 'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
 'sustainable_management_of_oceans_in_a_changing_climate']]

#area_1['urban_resilience'] = 0
area_2['small_island_dev_states_program_sids'] = 0
area_2['climate_resilient_access_to_water_supply_sanitation'] = 0
area_2['climate_resilient_integrated_water_resource_coastal_area_management'] = 0

Oceans_Water = Oceans_Water.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Oceans_Water = Oceans_Water.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Oceans_Water.iterrows():
    for i in oceans_and_water_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Oceans_Water.at[index, i] = 1
        else:
            Oceans_Water.at[index, i] = 0            
for i in oceans_and_water_programme:
    Oceans_Water= Oceans_Water.drop(columns=[i+"_x"])
    Oceans_Water= Oceans_Water.drop(columns=[i+"_y"])

Oceans_Water.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/oceans_water_full.xlsx")

## Environmental Governance

In [165]:
environmental_governance_programme = ['mainstreaming_green_economy', 'gcp_green_commodities_programme']

In [166]:
Environmental_Governance = Environmental_Governance[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'environmental_governance_programme']]

area_1 = technical_area_1[['PIMS_ID',  'gcp_green_commodities_programme']]

area_2 = technical_area_2[['PIMS_ID','mainstreaming_green_economy']]

area_1['mainstreaming_green_economy'] = 0
area_2['gcp_green_commodities_programme'] = 0

Environmental_Governance = Environmental_Governance.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Environmental_Governance = Environmental_Governance.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Environmental_Governance.iterrows():
    for i in environmental_governance_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Environmental_Governance.at[index, i] = 1
        else:
            Environmental_Governance.at[index, i] = 0            
for i in environmental_governance_programme:
    Environmental_Governance= Environmental_Governance.drop(columns=[i+"_x"])
    Environmental_Governance= Environmental_Governance.drop(columns=[i+"_y"])

Environmental_Governance.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/environmental_governance.xlsx")

'''full data'''
Environmental_Governance = result

Environmental_Governance = Environmental_Governance[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy', 'environmental_governance_programme']]

area_1 = technical_area_1[['PIMS_ID',  'gcp_green_commodities_programme']]

area_2 = technical_area_2[['PIMS_ID','mainstreaming_green_economy']]

area_1['mainstreaming_green_economy'] = 0
area_2['gcp_green_commodities_programme'] = 0

Environmental_Governance = Environmental_Governance.merge(area_1, how='left', left_on='PIMS_ID', right_on='PIMS_ID')
Environmental_Governance = Environmental_Governance.merge(area_2, how='left', left_on='PIMS_ID', right_on='PIMS_ID')

for index, row in Environmental_Governance.iterrows():
    for i in environmental_governance_programme:
        
        if row[i+"_x"] or row[i+"_y"]:
            Environmental_Governance.at[index, i] = 1
        else:
            Environmental_Governance.at[index, i] = 0            
for i in environmental_governance_programme:
    Environmental_Governance= Environmental_Governance.drop(columns=[i+"_x"])
    Environmental_Governance= Environmental_Governance.drop(columns=[i+"_y"])

Environmental_Governance.to_excel(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+"/environmental_governance_full.xlsx")

## Merge all data

In [167]:
df = Chemical_Waste.merge(CC_Adaption, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(Climate_Forest, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(Climate_Strategies, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(Ecosystems_Biodiversity, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(Energy_Programme, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(Oceans_Water, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(Environmental_Governance, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])

In [170]:
#df.to_csv(os.path.abspath(os.path.join('../..', 'data/processed/'))+"/technical_team.csv")